# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - проверить следующие гипотезы:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?


Таким образом, исследование пройдёт в четыре этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.
 4. Общий вывод.


## Шаг 1. Откройте файл с данными и изучите общую информацию

Для начала импортируем библиотеку `pandas`. Прочитаем файл `data.csv`' и сохраним результат в переменной `df`. Для ознакомления со структурой таблицы и представлением данных, выведем последние 15 строк.

In [1]:
import pandas as pd #импотируем Pandas
df = pd.read_csv('/datasets/data.csv') #читаем файл с данными и сохраняем в df
display(df.tail(15)) #показываем последние 15 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля
21511,0,-612.569129,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21512,0,-165.377752,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования
21513,0,-1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья
21514,0,-280.469996,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости


Согласно документацию имеем следующие столбцы:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

Выведем на экран общую информацию о таблице:

In [2]:
df.info() #получение общей информации о таблице

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21525 entries, 0 to 21524

Data columns (total 12 columns):

children            21525 non-null int64

days_employed       19351 non-null float64

dob_years           21525 non-null int64

education           21525 non-null object

education_id        21525 non-null int64

family_status       21525 non-null object

family_status_id    21525 non-null int64

gender              21525 non-null object

income_type         21525 non-null object

debt                21525 non-null int64

total_income        19351 non-null float64

purpose             21525 non-null object

dtypes: float64(2), int64(5), object(5)

memory usage: 2.0+ MB


Можно заметить, что таблица состоит из 12 столбцов, 21525 строк. В столбцах `days_employed` и `total_income` существуют пропуски данных.

Запросим для каждого столбца уникальные значения и их количество, для контроля на некорректные данные (за исключением `days_employed` и `total_income` так как они могут содержать множество значений) :

In [3]:
df['children'].value_counts() #получение уникальных значений и их количества столбца children

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце `children` можно заметить два параметра выбивающихся из общего ряда - это *20* и *-1*. О *-1* можно судить как об ошибочном вводе данных, так как количество детей не может быть отрицательным и налицо банальная опечатка. Также дела обстоят и со значением *20*, ведь если количество детей было бы истинно 20 (в количестве 76 единиц в выборке), то существовали было значения в промежутке от 6 до 20 с различными значениям. Но таких мы не наблюдаем, что наводит на мысль об ошибочном вводе.

В столбце `dob_years`, в отличии от других столбцов, запросим сортированные данные в виде списка, чтобы вычислить проблемы с указанием возраста:

In [4]:
df['dob_years'].sort_values().unique() #получение сортированных уникальных значений столбца dob_years

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

Также можем увидеть ошибку ввода данных на примере значений возраста 0. Но так как данные по возрасту не участвуют в нашем исследовании пропустим их. 

Получим уникальных значений и их количества столбца `education_id`:

In [5]:
df['education_id'].value_counts() #получение уникальных значений и их количества столбца education_id

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Каких-либо подозрительных данных не обнаружено, продолжим анализ дальше, получим уникальные значения столбца `family_status`:

In [6]:
df['family_status'].value_counts() #получение уникальных значений и их количества столбца family_status

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Здесь замечаний по данным нет, кроме значения *Не женат / не замужем*, на следующем этапе приведем данное значение к нижнему регистру как у других. Запросим данные столбца `family_status_id`:

In [7]:
df['family_status_id'].value_counts() #получение уникальных значений и их количества столбца family_status_id

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Замечаний нет, продолжаем анализ, смотрим значения в столбце `gender`:

In [8]:
df['gender'].value_counts() #получение уникальных значений и их количества столбца gender

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Здесь можно заметить, что существует одна запись с неуказанным полом, для нашего исследования оно не представляет значения, но тем, кто вносит данные следует на это обратить внимание. Продолжаем анализ в столбце `income_type`:

In [9]:
df['income_type'].value_counts()  #получение уникальных значений и их количества столбца income_type

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Подозрительных значений не наблюдаем, продолжаем нанализ столбца `debt`:

In [10]:
df['debt'].value_counts()  #получение уникальных значений и их количества столбца debt

0    19784
1     1741
Name: debt, dtype: int64

Тут тоже не наблюдается проблем, но можно заметить, что заемщики преимущественно (91,9%) не имели задолженности по возрату кредита. Посмотрим значения в столбце `education`:

In [11]:
df['education'].value_counts() #получение уникальных значений и их количества столбца education

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

В данном столбце, можно заметить проблемы с регистром, один и тот же уровень образования написан разным регистром, для простоты оперирования данным на следующем этапе приведем эти значения к единообразию.

Запросим данные по целям кредита с количеством таких целей:

In [12]:
df['purpose'].value_counts() #получение уникальных значений и их количества столбца purpose

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Как можно увидеть топ 3 целей займа на проведение бракосочетания, ну а затем идут операции с недвижимостью.

При визуальном анализе бросается в глаза, что в столбце `days_employed` у пенсионеров дни трудового стажа имеют положительное  и аномально высокое значения, проверим наличие положительных значений дней трудового стажа у других категорий

In [13]:
df.loc[(df.loc[:,'days_employed'] > 0) & (df.loc[:,'income_type'] != 'пенсионер') ]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,395302.838654,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


Не только у категории пенсионеров можно наблюдать аномальные значения, но и у безработных, следовательно можно сделать вывод, что положительные значения дней трудового стажа характерны для двух категорий: безработные и пенсионеры. Однако для нашего исследования данный столбец не имеет значения, тем не менее компетентным сотрудникам необходимо на это указать.

Проверим гипотезу об отсутствии одновременно в `days_employed` и `total_income`: 

In [14]:
columns_is_na = df[df['days_employed'].isna() & df['total_income'].isna()] #сделаем срез строк с пустыми значениями в columns_is_na
print('Количество пропусков значений одной строки в двух столбцах:',len(columns_is_na)) #выведем количество пропусков

Количество пропусков значений одной строки в двух столбцах: 2174


**Вывод**

Что получаем в итоге:

В таблице 21525 строк и 12 столбцов.

Нарушений в наименовании столбцов и типов данных в них - нет.

Гипотеза о одноврменном пропуске значений в `days_employed` и `total_income` подтвердилась. Так как значения в `total_income` важны для нашего исследования, заполним пропуски медианными значениями в дальнейших шагах. Но количество пропусков (10%) является достаточно большим, и в зависимости от целей исследования, объем потерянных данных может быть критическим, следует на это обратить внимание.

В столбце `children` было некорректно обозначено количество детей, в `dob_years` был указан нулевой возраст. Специалистам необходимо подумать над альтернативными способами ввода таких данных (например, экранная клавиатура) или добавить логические контроли, что повысит точность данных. 

В столбцах `family_status_id` и `education` проблема с регистром значений, рекомендуется сделать выпадающий список, чтобы специалист не вносил вручную данные, а выбирал из предложенных значений. 

Также можно заметить, что цели у заемщиков можно разделить на следующие группы:
* операции с недвижимостью (жильем)
* операции с автомобилем
* получение образования
* на проведение бракосочетания

Устраним проблемы в данных, необходимых для нашего исследования.

## Шаг 2. Предобработка данных

### Обработка пропусков

По предыдущему анализу видно, что в столбцах `days_employed` и `total_income` одинаковое количество пропусков данных - по 2174. Наличие значения ежемесячного дохода критично в нашем исследовании (такие потерянные данные составляют 10% от всех данных), заменим пустые значения на медианные для каждого типа вида занятости (такие же действия проделаем для количества дней стажа):

In [15]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median')) #заменим пустые значения на медианные для каждого типа занятости в total_income
df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('median')) #заменим пустые значения на медианные для каждого типа занятости в days_employed
display(df.info()) #проверим результаты

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21525 entries, 0 to 21524

Data columns (total 12 columns):

children            21525 non-null int64

days_employed       21525 non-null float64

dob_years           21525 non-null int64

education           21525 non-null object

education_id        21525 non-null int64

family_status       21525 non-null object

family_status_id    21525 non-null int64

gender              21525 non-null object

income_type         21525 non-null object

debt                21525 non-null int64

total_income        21525 non-null float64

purpose             21525 non-null object

dtypes: float64(2), int64(5), object(5)

memory usage: 2.0+ MB


None

А теперь разберемся с данными в столбце `days_employed`. Вы определенно знаем, что такие данные больше 0, посмотрим начальное и конечное значение, для этого запросим сортированный массив по возрастанию:

In [16]:
df[df['days_employed']>0]['days_employed'].sort_values().unique() #запросим сортированный массив, с данными больше 0 в столюце стажа

array([328728.72060452, 328734.92399633, 328771.3413868 , ...,
       401675.09343386, 401715.81174889, 401755.40047533])

Начинается с 328 728 и заканчиваются на 401 755, а что если произошел сбой, и эти данные не в днях, а в часах? Попробуем првоерить такую гипотезу и разделим такие положительные значения на 24:

In [17]:
df.loc[df['days_employed'] > 0, 'days_employed'] = df['days_employed']/24 #разделим положительные значения в столбце days_employed на 24 часа

Проверим значения в стоблце снова, таким же методом в сортированном массиве:

In [18]:
df[df['days_employed']>0]['days_employed'].sort_values().unique() #снова получим сортированный массив для контроля результатов

array([13697.03002519, 13697.28849985, 13698.80589112, ...,
       16736.46222641, 16738.15882287, 16739.80835314])

13697 разделим на 365 и получим 37 лет, что выглядит правдоподобно. А теперь приведем отричательные значения в положительные:

In [19]:
df['days_employed'] = df['days_employed'].abs() #представим все значения в столбце days_employed в положительном виде
display(df.head(5)) #вывыдем первые 5 строк таблицы, для контроля результата

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

Заменили пустые значения в `total_income`, но не одним медианным значениям для всех (ведь у каждого типа занятости своя заработная плата), а медианным значениям для каждого типа занятости. Также поступили с данными в стоблце `days_employed`, но помимо этого разобрались с аномалией высоких дней стажа (которые оказались часами) и привели отрицательные значения в положительные и  можем продолжить наше исследование дальше.

### Замена типа данных

Для дальнейшего анализа и лучшего восприятия данных (большое количество знаков после запятой не влияет на исследование), изменим тип данных в столбцах `days_employed` и `total_income` с float на int. Так как это позволит проще проводить различные операции над данными. Используется метод `astype`, так как оба этих столбца имеют данные *float64*, а не *object* и ошибок при конвертации быть не должно (и не потребуется их обработка методом **try except**).

In [21]:
df['days_employed'] = df['days_employed'].astype('int') #переведем значения в days_employed с float в int
df['total_income'] = df['total_income'].astype('int') #переведем значения в total_income с float в int
display(df.head(10)) #отобразим первые 10 строк для контроля правильного выполпнения кода

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

В этом шаге мы преобразовали данные из float64 в int в столбцах `days_employed` и `total_income`, с такими данными легче будет производить различные манипуляции и подсчеты и визуально они не оттягощают таблицу.


### Обработка дубликатов

При анализе , мы увидели, что в столбце `children` можно заметить два параметра выбивающихся из общего ряда - это 20 и -1. Приведем эти некорректные значения в корректные, и запросим для проверки уникальные значения и их количество.

In [22]:
df.loc[df['children'] == -1,'children'] = 1 #исправляем значение -1 на правильное в стоблце children
df.loc[df['children'] == 20,'children'] = 2 #исправляем значение 20 на правильное в стоблце children
df['children'].value_counts()  #получим количество уникальных значений и их количество в стоблце children

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

При первичном знакомстве с таблицей, в столбце `education` мы выявили одинаковые данные, но записанные разным регистром. Исправим это, приведем все виды написания значений к нижнему регистру и выполним проверку, запросив уникальные значения и их количество:

In [23]:
df['education'] = df['education'].str.lower() #переведем значения в столбце education к нижнему регистру
df['education'].value_counts() #получим количество уникальных значений и их количество в стоблце education

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Также поправим значение *Не женат / не замужем*  в столбце `family_status`, чтобы его написание соответствеовало другим:

In [24]:
df['family_status'] = df['family_status'].str.lower() #переведем значения в столбце education к нижнему регистру
df['family_status'].unique() #выведем массив уникальных значений

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

Значение *Не женат / не замужем*, как и другие теперь в нижнем регистре. Проверим всю таблицу на наличие явных дубликатов:

In [25]:
df.duplicated().sum() #запрашиваем количество дубликатов строк в массиве

71

71 дублирующая строка - 0,33% от всего массива данных. В рамках нашего исследования это очень небольшая величина, и ей можно пренебречь. Удалим дублирующие строки, сбросим индексацию строк и заново запросим количество дублей:

In [26]:
df = df.drop_duplicates().reset_index(drop = True) #удаляем дубликаты строк и сбрасываем индексацию у таблицы
df.duplicated().sum() #запрашиваем количество дубликатов строк в массиве

0

**Вывод**

Воспользовавшись методом приведения значений к нижнему регистру, нам удалось уменьшить количество записей одного и того же параметра, благодаря этому улучшилось восприятие и читаемость таблицы. 

Опеределили дубликаты строк, удалили их. Такие дубликаты возможны, когда данные не привязаны к ID пользователя. Например в один период времени человек брал заем для одних целей (возраст был, предположим 30 лет), а позже, когда возраст изменился - для других. Следует взять этот нюанс в разработку.

Продолжим исследование дальше. 

### Лемматизация

Цели у заемщиков в большинстве одинаковые, но как мы увидели в первичном анализе, сформулированы и записаны по-разному. Чтобы привести их к "общему знаменателю" из первичного анализа сформируем слова-ключи, напишем функцию, которая будет искать ключевые слова и затем создадим новый стобец с логически правильно сформулированной целью заемщика.

In [27]:
from pymystem3 import Mystem #импортируем библиотеку для лемматизации
m = Mystem()
'''Преобразуем цели с помощью библиотеки PyMystem, она лемматизирует слова в список слов-"инфинитивов".
 В полученном списке найдем ключевое слово, которое нас интересует, и вернет корректное название категории'''
def purpose_change(purpose):
    row = m.lemmatize(purpose)
    for aim in row:
        if 'автомобиль' in aim:
            return 'операции с автомобилем'
        if ('жилье' in aim) or ('недвижимость' in aim):
            return 'операции с недвижимостью'
        if 'образование' in aim:
            return 'получение образования'
        if 'свадьба' in aim:
            return 'бракосочетание'

df['purpose_def'] = df['purpose'].apply(purpose_change) #применим нашу функцию для поиска цели и создадим новый столбец purpose_def
display(df['purpose_def'].value_counts()) #получим список с общими целями и их количеством в столбце purpose_def

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
бракосочетание               2324
Name: purpose_def, dtype: int64

**Вывод**

Преобразовав по ключевым словам цели заемщиков, нам удалось избежать повторений и тем самым количество вариаций целей уменьшилось с 38 до 4 основных. Рекомендуется при внесении данных, создать список с формализованными целями, чтобы избежать искажений в формулировке и двусмысленности.

### Категоризация данных

Но вернемся к главным вопросам нашего анализа. Первый звучит так **Есть ли зависимость между наличием детей и возвратом кредита в срок?**. 

В нашей таблице есть сведения о количестве детей, сформируем два варианта ответа на этот вопрос, первый - наличие детей и отсутствие вместе с задолженностью, второй - зависимость количества детей и задолженности. 

С помощью функции ответим на первую часть, пусть такая функция будет возвращать сведения о наличии детей, если в соответствующем столбце `children` значение больше 1. И пусть все эти значения будут оформлены в отдельный столбец `ch_available`:

In [28]:
def credit_return_child(row):
    '''Функция принимает значение столбца children, если оно положительно, то возращает
    значение о наличии детей и наоборот'''
    if row['children'] == 0:
        return 'детей нет'
    if row['children'] > 0:
        return 'дети есть'
df['ch_available'] = df.apply(credit_return_child, axis = 1) #объявим столбец 'ch_available' куда пойдут итоги работы функции
display(df.head(5)) #отобразим первые 5 строк для контроля работы функции

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_def,ch_available
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью,дети есть
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем,дети есть
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью,детей нет
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,получение образования,дети есть
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,бракосочетание,детей нет


Теперь столбец `ch_available` позволит сделать соответствующую группировку и ответить на этот вопрос. Таким же способом "отфильтруем" заработную плату, чтобы ответить на третий вопрос, где фигурирует уровень дохода. Но в описании к таблице не сказано о какой валюте или стране идет речь, на какие группы делить работающих. <a href='https://zavtra.ru/blogs/eto_interesno_i_polezno_znat_'>Деление на группы мы посмотрим здесь, их будет 5:</a> 
1. Нищие

2. Бедные

3. Выше бедности

4. Средний класс

5. Высший класс (богатые и сверхбогатые)

Определим уровни заработной платы, чтобы отнести работников в определенному классу, для этого воспользуемся функцией `qcut()`

In [29]:
pd.qcut(x=df['total_income'], q=5)

0        (214618.2, 2265604.0]
1          (98537.6, 132134.4]
2         (132134.4, 161335.0]
3        (214618.2, 2265604.0]
4         (132134.4, 161335.0]
                 ...          
21449    (214618.2, 2265604.0]
21450     (132134.4, 161335.0]
21451     (20666.999, 98537.6]
21452    (214618.2, 2265604.0]
21453     (20666.999, 98537.6]
Name: total_income, Length: 21454, dtype: category
Categories (5, interval[float64]): [(20666.999, 98537.6] < (98537.6, 132134.4] < (132134.4, 161335.0] < (161335.0, 214618.2] < (214618.2, 2265604.0]]

Соответственно получаем:
1. Нищие - до 98662 у.е.

2. Бедные - от 98663 до 132142 у.е.

3. Выше бедности - от 132143 до 161151 у.е.

4. Средний класс - от 161152 до 214269 у.е.

5. Высший класс (богатые и сверхбогатые) - свыше 214270 у.е.

С помощью фукнции обработаем значения заработной платы и вернем сведения с градацией в столбец `income_level`:

In [30]:
def credit_return_income(income):
    '''Функция принимает значение столбца total_income, и в зависимости от уровня, 
    возращает соответствующее значение в столбец income_level '''
    if income <= 98662:
        return 'нищие'
    if 98663 < income  <= 132142:
        return 'бедные'
    if 132143 < income <= 161151:
        return 'выше бедности'
    if 161152 < income <= 214269:
        return 'средний класс'
    if income  > 214270:
        return 'высший класс'
df['income_level'] = df['total_income'].apply(credit_return_income) #объявим столбец 'income_level' куда пойдут итоги работы функции
display(df.head(5)) #отобразим первые 5 строк для контроля работы функции

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_def,ch_available,income_level
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью,дети есть,высший класс
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем,дети есть,бедные
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью,детей нет,выше бедности
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,получение образования,дети есть,высший класс
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,бракосочетание,детей нет,выше бедности


In [31]:
df['income_level'].value_counts()

высший класс     4305
нищие            4305
средний класс    4298
бедные           4278
выше бедности    4267
Name: income_level, dtype: int64

**Вывод**

А теперь попробуем ответить на поставленные вопросы. 

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на этот выведем для наглядности таблицу *а* из предыдущих шагов:

In [32]:
round(df.groupby('ch_available')['debt'].mean()*100,2) #выведем на экран данные по наличию детей/задолженность

ch_available
детей нет    7.54
дети есть    9.21
Name: debt, dtype: float64

В разрезе присутствия в семье детей и их отсутствия, можно сказать следующее:
* Процент наличия задолженности оп возврату займа, в семьях где **есть** дети составляет - 9,21%
* Процент наличия задолженности оп возврату займа, в семьях где **нет** детей - 7,54%
Но значит ли, что наличие детей заставляет копить задолженность? Может быть в семьях с разным количеством детей, эти показатели разные? 

Проверим это:

In [33]:
round(df.groupby('children')['debt'].mean()*100,2) #выведем на экран данные по количеству детей/задолженность

children
0    7.54
1    9.17
2    9.49
3    8.18
4    9.76
5    0.00
Name: debt, dtype: float64

Тут можно наблюдать следующую картину:
* в семьях с 1 ребенком, процент задолженности в своей группе составляет 9,17%
* с 2мя детьми, такой процент чуть выше - 9,49%
* с 3мя детьми, мы должны были увидеть небольшой рост (как в двух категориях выше), но тут падение на 1,3% и такой процент составляет 8,18%
* в семьях с 4мя детьми, процент задолженности снова выше, и выше он всех предудщих категорий - 9,76%
* ну а самая неождианность - семьи с 5ю детьми. У таких семей задолженность отсутствует вприципе.

**Вывод**

В целом, наличие детей влияет на появление задолженности по займу. И чем больше детей, тем выше этот процент становится. Но у нас есть абсолютно выбивающийся показатель семей с 5ю детьми, как так вышло? Тут можно оценить добросовестность заемщиков. Но не стоит забывать, таких заемщиков в выборке всего 9 и появление одной задолженности, резко увеличит этот процент с 0 до 10%, тем самым еще раз подтвердя нашу гипотезу.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [34]:
round(df.groupby('family_status')['debt'].mean()*100,2) #выведем на экран данные по семейному положению/задолженность

family_status
в разводе                7.11
вдовец / вдова           6.57
гражданский брак         9.35
женат / замужем          7.55
не женат / не замужем    9.75
Name: debt, dtype: float64

**Вывод**

Начнем от выского к низкому:
* самый высокий процент задолженности в группе *не женат / не замужем* - 9,75%
* незначительно меньше у людей в *гражданском браке* - 9,35%
* ниже процент становится в группе *женат / замужем* - 7,55%
* немного меньше у тех, кто *в разводе* - 7,11%
* ну и самая низкая задолженность у *вдовцов* - 6,57%

Можно заметить закономерность, что у людей после вступления в брак (имеется ввиду и люди в нем состоявшие) процент задолженности значительно падает, и их можно рассматривать как благонадежных заемщиков.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [35]:
round(df.groupby('income_level')['debt'].mean()*100.,2) #выведем на экран данные по уровеню дохода/задолженность

income_level
бедные           8.42
высший класс     7.02
выше бедности    8.76
нищие            8.01
средний класс    8.38
Name: debt, dtype: float64

**Вывод**

В разрезе классов, мы получили следующие результаты :
1. Нищие - 8,01%

2. Бедные - 8,42%

3. Выше бедности- 8,76%

4. Средний класс - 8,38%

5. Высший класс (богатые и сверхбогатые) - 7,02%

Можно заметить, что люди в классе "нищие" и "выскшие класс" стараются избегать задолженности *(она у них в группе составляет 8,01% и 7,02% соответственно)*. Самая высокая задолженность в классе "выше бедности" *(8,76%)*. К среднему уровню задолженности можно отнести класс "бедные" - 8,42% и "средний класс" - 8,38%.

Можно рассматривать как более благонадженых заемщиков людей в высшем классе, менее - в классе "выше бедности". Однако, можно отметить, что практически все значения (в четрыех группах из 5) колеблются в районе 8%, это связано с тем, что Заказчик не передал для анализа свой вариант разделения на классы по уровню заработной платы. И по сути мы усреднили эти 5 групп по количеству человек в нихи на вызоде получили те же самые "средние" результаты у всех.

- Как разные цели кредита влияют на его возврат в срок?

Также обратимся к сводной таблице *d* из предыдущего шага:

In [36]:
round(df.groupby('purpose_def')['debt'].mean()*100,2) #выведем на экран данные по цели кредита/задолженность

purpose_def
бракосочетание              8.00
операции с автомобилем      9.36
операции с недвижимостью    7.23
получение образования       9.22
Name: debt, dtype: float64

**Вывод**

Пойдем по нисходящей:
* большой процент задолженности в группе *операции с автомобилем* - 9,36%
* затем идет группа *получение образования* - 9,22%
* на *бракосочетание* приходится - 8%
* а на *операции с недвижимостью* - 7,23%

Как можно увидеть наиболее рискованные, с точки зрения задолженности, являются операции с автомобилем, а наименее - операции с недвижимостью.

In [37]:
df.groupby('family_status')['debt'].mean()*100

family_status
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.347145
женат / замужем          7.545182
не женат / не замужем    9.750890
Name: debt, dtype: float64

## Шаг 4. Общий вывод

Сделав предвариательный анализ и рассмотрев исходные данные, можем отметить, что существует проблема наполнения данных и качества. В кредитном отделе на этапе ввода необходимо  сделать логические контроли, чтобы не мог был возраст человека нулевым, цели кредита и семейное положение были формализованы и не требовали набора вручную. Стоит обратить внимание на выгрузку данных, так как в наше датасете были потеряны 10% данных о стаже и заработной плате, ведь эти проценты могут легко увеличиться, и исследование уже не будет корректным, будет потрачено время на восстановление данных.

Что касается наших вопросов, несомненно каждый аспект влияет на появление задолженности. Выведем сводную таблицу оценки портрета заемщика, таких портретов будет 5. Где 1 - самый худший вариант (вероятно возникнет задолженность), а 5 - самый лучший вариант (маловероятно, что возникнет задолженность):


In [38]:
data_research = [['Количество детей', '4 ребенка и более', '2 ребенка', '1 ребенок', '3 ребенка', 'детей нет'],
          ['Семейное положение', 'не женат / не замужем', 'граждаский брак', 'женат / замужем', 'в разводе', 'вдовец / вдова'],
         ['Класс по заработку','выше бедности','бедные','средний класс','нищие','высший класс'],
         ['Цель займа','операции с автомобилем','операции с автомобилем','получение образования','бракосочетание ','операции с недвижимостью']]
columns_end = ['Параметры', '1 (хуже)','2','3','4','5 (лучше)']
conclusion = pd.DataFrame(data = data_research, columns = columns_end)
display(conclusion)

,Параметры,1 (хуже),2,3,4,5 (лучше)
0,Количество детей,4 ребенка и более,2 ребенка,1 ребенок,3 ребенка,детей нет
1,Семейное положение,не женат / не замужем,граждаский брак,женат / замужем,в разводе,вдовец / вдова
2,Класс по заработку,выше бедности,бедные,средний класс,нищие,высший класс
3,Цель займа,операции с автомобилем,операции с автомобилем,получение образования,бракосочетание,операции с недвижимостью


Однако следует заметить, что для получения более объективного анализа, от Заказчика необходимо получить критерии по уровню заработной платы, чтобы точно определить класс заемщика. И тогда конечный итог может несколько другим.